In [1]:
import asyncio
import header
from src import gs
from src.webdriver import Driver, Firefox
from src.utils.printer import pprint as print
from src.logger import logger

In [2]:
d = Driver(Firefox)

2025-04-08 06:41:23,255 - INFO - ℹ️ Starting Firefox WebDriver 
2025-04-08 06:41:33,626 - INFO - ℹ️ Browser started successfully, window_mode='normal' 


Поставщик (`supplier`): `KSP` 

Категория (`category`): Память для компьютера. 

`id_category` = `3776`

[see RAM category in store](https://emil-design.com/__ooOOoo__/index.php/sell/catalog/categories/4768?token=82ad575c3e6bb9c4808df52810fc5250)

In [4]:
d.get_url("https://ksp.co.il/mob/cat/1038")

True

In [5]:
d.scroll(8)

True

In [6]:
product_links_locator:dict =  {
    "attribute": "href",
    "by": "XPATH",
    "selector": "//a[contains(@href, '/item/')]",
    "if_list":"all",
    "use_mouse": False, 
    "mandatory": True,
    "timeout":0,
    "timeout_for_event":"presence_of_element_located",
    "event": None
}

In [ ]:
product_links:list | str = await( d.execute_locator(product_links_locator) ) 
product_links:list = product_links if isinstance(product_links, list) else [product_links]

In [ ]:
logger.info(f'returns type:{type(product_links)}')
logger.info(f'links count:{len(product_links)}')
print(product_links)

In [ ]:
product_name_locator: dict = {
    "attribute": "aria-label",
    "by": "XPATH",
    "selector": "//div[@id = 'product-page-root']//h1[contains(@class, 'title')]",
    "if_list": "first",
    "use_mouse": False,
    "timeout": 1,
    "timeout_for_event": "presence_of_element_located",
    "event": None,
    "mandatory": True,
    "locator_description": "KSP - Название товара"
  }

In [ ]:
product_name_mobile_locator: dict =  {
    "attribute": "innerText",
    "by": "XPATH",
    "selector": "//h1[@class = 'rtl-16esk6i']",
    "if_list": "first",
    "use_mouse": False,
    "timeout": 2,
    "timeout_for_event": "presence_of_element_located",
    "event": None,
    "mandatory": True,
    "locator_description": "KSP mobile version - Название товара"
  }

In [ ]:
async def get_product_name(d:Driver) -> str:
    """Функция возвращает значение поля `name` товара.
    В зависимости от версии страницы `/mob/` | `/web/` (мобильная / десктопная) 
    функция подставляет релевантный локатор"""
    
    if '/mob/' in d.current_url:   
        return await d.execute_locator(product_name_mobile_locator)
    else:
        return await d.execute_locator(product_name_locator)
        

In [ ]:
counter:int = 0 # <- счетчик, чтобы сервер нас не забанил за частые запросы
for url in product_links:
    counter += 1
    d.get_url(url)
    name = await get_product_name(d)
    print(f'Product name {name}')
    if counter > 5:
        logger.debug('stopped by debugger', None, False)
        return
    d.wait(3)